#### Project 5: Predicted Pollution Mortality
#### Corey J Sinnott
# Model Testing

## Executive Summary

This report was commissioned to explore mortality influenced by pollution. Data was obtained from several sources listed below. The problem statement was defined as, can we predict pollution mortality? After in-depth analysis, conclusions and recommendations will be presented.


## Contents:
- [Model Testing](#Model-Testing)



#### Importing Libraries

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, plot_confusion_matrix, classification_report, plot_roc_curve
import statsmodels.api as sm
from category_encoders import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, SGDClassifier, SGDRegressor, ElasticNet, LassoLars
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import make_column_transformer, make_column_selector, TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn import set_config
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE, RFECV, VarianceThreshold
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix 
import scipy as sp
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

#### Reading in Data

In [7]:
df = pd.read_csv('./data/model_df.csv')

In [8]:
df.sample(3)

,Year,annual_co2_emmissions,health_spend_per_capita,life_expectancy,ozone_depleting_emissions,min_daily_ozone,mean_daily_ozone,population,pollution_deaths
2827,1991,373.793321,NaN,70.587805,68.67,94.0,119.0,38083000.0,26374.98
3270,2001,0.529084,NaN,49.631000,15.97,91.0,100.9,6448000.0,1009.41
161,2011,4.968878,252.650423,73.571000,3.08,95.0,106.5,2877000.0,2241.88


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4015 entries, 0 to 4014
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year                       4015 non-null   int64  
 1   annual_co2_emmissions      4010 non-null   float64
 2   health_spend_per_capita    2793 non-null   float64
 3   life_expectancy            3806 non-null   float64
 4   ozone_depleting_emissions  3868 non-null   float64
 5   min_daily_ozone            3874 non-null   float64
 6   mean_daily_ozone           3874 non-null   float64
 7   population                 4015 non-null   float64
 8   pollution_deaths           4015 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 282.4 KB


In [10]:
df['crude_death_per_1_000_000'] = (df['pollution_deaths'] / df['population'] * 1_000_000)

# Model Testing

#### Modeling with population and crude death rate
 - Unsuccessful; would have to standardize all of the variables to the same scale, which may introduce too much error.

In [12]:
X = df.drop(['health_spend_per_capita', 'pollution_deaths', 
             'crude_death_per_1_000_000', 'life_expectancy'], axis = 1)
y = df['crude_death_per_1_000_000']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [14]:
impute = SimpleImputer(missing_values = np.nan)

In [15]:
X_train_fill = impute.fit_transform(X_train)
X_test_fill = impute.transform(X_test)

In [16]:
ss = StandardScaler()

In [17]:
X_train_fill_scaled = ss.fit_transform(X_train_fill)
X_test_fill_scaled = ss.transform(X_test_fill)

In [18]:
pf = PolynomialFeatures()

In [19]:
X_train_use = pf.fit_transform(X_train_fill)
X_test_use = pf.transform(X_test_fill)

In [20]:
LL = LassoLars()

In [21]:
LL.fit(X_train_use, y_train)

LassoLars()

In [22]:
y_pred = LL.predict(X_test_use)

In [23]:
def regression_eval(y_test, y_pred):
    print(f'MSE = {np.round(mean_squared_error(y_test, y_pred), 3)}')
    print(f'RMSE = {np.round(mean_squared_error(y_test, y_pred, squared = False), 3)}')
    print(f'MAE = {np.round(mean_absolute_error(y_test, y_pred), 3)}')
    print(f'r^2  = {np.round(r2_score(y_test, y_pred), 3)}')

regression_eval(y_test, y_pred)

MSE = 43403.047
RMSE = 208.334
MAE = 161.007
r^2  = -0.001


#### Modeling with population and total deaths
 - Population introducing a lot of colinearity

In [25]:
X = df.drop(['health_spend_per_capita', 'pollution_deaths', 'crude_death_per_1_000_000'], axis = 1)
y = df['pollution_deaths']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [27]:
impute = SimpleImputer(missing_values = np.nan)

In [28]:
X_train_fill = impute.fit_transform(X_train)
X_test_fill = impute.transform(X_test)

In [29]:
ss = StandardScaler()

In [30]:
X_train_fill_scaled = ss.fit_transform(X_train_fill)
X_test_fill_scaled = ss.transform(X_test_fill)

In [31]:
pf = PolynomialFeatures()

In [32]:
X_train_use = pf.fit_transform(X_train_fill)
X_test_use = pf.transform(X_test_fill)

In [33]:
LL = LassoLars()

In [34]:
LL.fit(X_train_use, y_train)

LassoLars()

In [35]:
y_pred = LL.predict(X_test_use)

In [36]:
def regression_eval(y_test, y_pred):
    print(f'MSE = {np.round(mean_squared_error(y_test, y_pred), 3)}')
    print(f'RMSE = {np.round(mean_squared_error(y_test, y_pred, squared = False), 3)}')
    print(f'MAE = {np.round(mean_absolute_error(y_test, y_pred), 3)}')
    print(f'r^2  = {np.round(r2_score(y_test, y_pred), 3)}')

regression_eval(y_test, y_pred)

MSE = 86204777.656
RMSE = 9284.653
MAE = 3964.363
r^2  = 0.991


#### Modeling without population and total deaths

In [38]:
X = df.drop(['health_spend_per_capita', 'pollution_deaths',
             'crude_death_per_1_000_000', 'population'], axis = 1)
y = df['pollution_deaths']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [40]:
impute = SimpleImputer(missing_values = np.nan)

In [41]:
X_train_fill = impute.fit_transform(X_train)
X_test_fill = impute.transform(X_test)

In [42]:
ss = StandardScaler()

In [43]:
X_train_fill_scaled = ss.fit_transform(X_train_fill)
X_test_fill_scaled = ss.transform(X_test_fill)

In [44]:
pf = PolynomialFeatures()

In [45]:
X_train_use = pf.fit_transform(X_train_fill)
X_test_use = pf.transform(X_test_fill)

In [46]:
LL = LassoLars()

In [47]:
LL.fit(X_train_use, y_train)

LassoLars()

In [48]:
y_pred = LL.predict(X_test_use)

In [49]:
def regression_eval(y_test, y_pred):
    print(f'MSE = {np.round(mean_squared_error(y_test, y_pred), 3)}')
    print(f'RMSE = {np.round(mean_squared_error(y_test, y_pred, squared = False), 3)}')
    print(f'MAE = {np.round(mean_absolute_error(y_test, y_pred), 3)}')
    print(f'r^2  = {np.round(r2_score(y_test, y_pred), 3)}')

regression_eval(y_test, y_pred)

MSE = 2044458913.337
RMSE = 45215.693
MAE = 13576.792
r^2  = 0.765


In [50]:
#XGBoost

In [53]:
GB = GradientBoostingRegressor()

In [54]:
GB.fit(X_train_use, y_train)

GradientBoostingRegressor()

In [55]:
gb_preds = GB.predict(X_test_use)

In [57]:
def regression_eval(y_test, y_pred):
    print(f'MSE = {np.round(mean_squared_error(y_test, y_pred), 3)}')
    print(f'RMSE = {np.round(mean_squared_error(y_test, y_pred, squared = False), 3)}')
    print(f'MAE = {np.round(mean_absolute_error(y_test, y_pred), 3)}')
    print(f'r^2  = {np.round(r2_score(y_test, y_pred), 3)}')

regression_eval(y_test, gb_preds)

MSE = 702159637.87
RMSE = 26498.295
MAE = 6881.397
r^2  = 0.919


In [ ]:
#RandomForest

In [58]:
RF = RandomForestRegressor()

In [59]:
RF.fit(X_train_use, y_train)

RandomForestRegressor()

In [60]:
rf_preds = RF.predict(X_test_use)

In [61]:
def regression_eval(y_test, y_pred):
    print(f'MSE = {np.round(mean_squared_error(y_test, y_pred), 3)}')
    print(f'RMSE = {np.round(mean_squared_error(y_test, y_pred, squared = False), 3)}')
    print(f'MAE = {np.round(mean_absolute_error(y_test, y_pred), 3)}')
    print(f'r^2  = {np.round(r2_score(y_test, y_pred), 3)}')

regression_eval(y_test, rf_preds)

MSE = 418577947.842
RMSE = 20459.178
MAE = 5699.199
r^2  = 0.952


Lasso with least angle regression shows best potential to predict mortality given the covariate nature of the variables. Will proceed with a GridSearch to tune, and further exploration of features.